In [2]:
import numpy as np
import cv2 as cv
from os import path
from tqdm import trange
import matplotlib.pyplot as plt

In [3]:
TRAIN_DIR = "labeled"
video_file = "0.hevc"
text_file = "0.txt"


In [4]:
# convert text file to array
pitches, yaws = [], []
with open(path.join(TRAIN_DIR, text_file), "r") as f:
     for line in f:
        pitch, yaw = line.split()
        pitches.append(float(pitch))
        yaws.append(float(yaw))



In [5]:
# cap = cv.VideoCapture(path.join(TRAIN_DIR, video_file))           

In [6]:
# params for ShiTomasi corner detection
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Parameters for lucas kanade optical flow
lk_params = dict( winSize  = (15, 15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))
# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Video params
FPS = 20
num_frames = 20*60

In [7]:
# cap = cv.VideoCapture(path.join(TRAIN_DIR, video_file))
# ret, frame1 = cap.read()
# prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
# hsv = np.zeros_like(frame1)
# hsv[..., 1] = 255

# magnitudes = []
# angles = []

# t = trange(num_frames)

# for epoch in t:
#     ret, frame2 = cap.read()
#     if not ret:
#         print('No frames grabbed!')
#         break
#     next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
#     flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
#     mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])

#     magnitudes.append(mag.flatten())
#     angles.append(ang.flatten())
#     hsv[..., 0] = ang*180/np.pi/2
#     hsv[..., 2] = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)
#     bgr = cv.cvtColor(hsv, cv.COLOR_HSV2BGR)
#     # cv.imshow('frame2', bgr)
#     # k = cv.waitKey(30) & 0xff
#     # if k == 27:
#     #     break
#     # elif k == ord('s'):
#     #     cv.imwrite('opticalfb.png', frame2)
#     #     cv.imwrite('opticalhsv.png', bgr)
#     prvs = next
# # cv.destroyAllWindows()

In [8]:
# save magnitude and angles

import pickle
# with open("magnitudes.pkl", "wb") as f:
#     pickle.dump(magnitudes, f)

    
# with open("angles.pkl", "wb") as f:
#     pickle.dump(angles, f)


with open("magnitudes.pkl", "rb") as f:
    magnitudes = pickle.load(f)

with open("angles.pkl", "rb") as f:
    angles = pickle.load(f)
    

In [10]:
mag_ang = []
for i, el in enumerate(magnitudes):
    mag_ang.append(np.concatenate((el, angles[i])))

In [9]:
mod_pitches = pitches[1:]
mod_yaws = yaws[1:]

In [10]:
from sklearn.model_selection import train_test_split
X_mag_train, X_mag_test = train_test_split(magnitudes, test_size=0.2, random_state=25)
X_ang_train, X_ang_test = train_test_split(angles, test_size=0.2, random_state=25)

Y_pitch_train, Y_pitch_test = train_test_split(mod_pitches, test_size=0.2, random_state=25)
Y_yaw_train, Y_yaw_test = train_test_split(mod_yaws, test_size=0.2, random_state=25)


In [12]:
X_train = np.column_stack((X_mag_train, X_ang_train))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

regr = RandomForestRegressor(max_depth=4, random_state=0)
regr.fit(X_train, Y_pitch_train)


In [ ]:
from sklearn.metrics import mean_squared_error

X_test = np.column_stack(X_mag_test, X_ang_test)

Y_pitch_pred = regr.predict(X_test)
print(mean_squared_error(Y_pitch_test, Y_pitch_pred)

In [15]:

# input dim is mag and ang flattened added together
# (874 * 1164)*2

# input_dim = (874 * 1164)*2

# output_dim = 1
# import torch
# import torch.nn as nn



# class PitchNet(nn.Module):
#     def __init__(self):
#         super(Network, self).__init__()
#         self.l1 = nn.Linear(input_dim, hidden_layers)
#         self.act = nn.ReLU()
#         self.l2 = nn.Linear(hidden_layers, output_dim)

#     def forward(self, x):
#         x = self.l1(x)
#         x = self.act(x)
#         x = self.l2(x)
#         return x

# model = PitchNet()
# loss_function = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters())



# net = PitchNet()
# print(net)

NameError: name 'yaws' is not defined

In [ ]:
# epochs = 2
# BS = 32

# t = trange(epochs)

# losses, accuracies = [], []

# for epoch in t:
#     samp = np.random.randint(0, X_train.shape[0], size=(BS))

#     X = torch.tensor(X_train[samp]).float()
#     Y = torch.tensor(Y_train[samp]).long()

#     optimizer.zero_grad()
    
#     # forward propogation
#     outputs = model(X)
    
#     cat = torch.argmax(outputs, dim=1)
#     accuracy = (cat == Y).float().mean()
    
#     loss = loss_function(outputs, Y)
#     loss.backward()
#     optimizer.step()
    
#     loss, accuracy = loss.item(), accuracy.item()
#     losses.append(loss)
#     accuracies.append(accuracy)
    
        
#     t.set_description(f"loss {loss:.2f} accuracy {accuracy:.2f}")

# print(f"loss {loss:.2f} accuracy {accuracy:.2f}")
# plt.ylim(-1, 2)
# plt.plot(losses, label="loss")
# plt.plot(accuracies, label="accuracy")
# plt.legend()